In [1]:
! git clone https://github.com/NicolasAG/SGinPG.git

Cloning into 'SGinPG'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 102 (delta 46), reused 90 (delta 40), pack-reused 0
Receiving objects: 100% (102/102), 856.37 KiB | 7.08 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [2]:
%cd SGinPG/data

/Users/amirhosein/Development/PycharmProjects/len_gen/scripts/dataset_builders/SGinPG/data


In [3]:
! chmod a+x setup.sh
! ./setup.sh

downloading clutrr data files...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2272    0  2272    0     0   8548      0 --:--:-- --:--:-- --:--:--  8875
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  333M  100  333M    0     0  32.7M      0  0:00:10  0:00:10 --:--:-- 34.4M
unzipping...
done.


In [4]:
! ls

backward             relations_store.yaml setup.sh
forward              rules_store.yaml


In [4]:
import re
p = re.compile(r'<STORY>((.)+)<QUERY>((.)+)<PROOF>((.)+)<ANSWER>((.)+)')
def process_instance(instance: str):
    res =  re.search(p, instance)
    story = res.group(1).strip()
    query = res.group(3).strip()
    proof = res.group(5).strip()
    answer = res.group(7).strip()
    source = f"{story} [query] {query}"
    target = answer.replace(".", "").strip()
    scratchpad = [s.strip().replace("Since", "since") for s in proof.split(".") if len(s.strip()) > 0]
    scratchpad = scratchpad
    return {
        "source": source,
        "target": target,
        "scratchpad": scratchpad,
    }

s = "<STORY> ent_3 is a daughter of ent_6 . ent_6 is a husband to ent_20 . <QUERY> How is ent_20 related to ent_3 ? <PROOF> Since ent_3 is a daughter of ent_6 , and ent_6 is a husband to ent_20 , then ent_3 is the daughter of ent_20 . <ANSWER> ent_3 is the daughter of ent_20 ."

process_instance(s)

{'source': 'ent_3 is a daughter of ent_6 . ent_6 is a husband to ent_20 . [query] How is ent_20 related to ent_3 ?',
 'target': 'ent_3 is the daughter of ent_20',
 'scratchpad': ['since ent_3 is a daughter of ent_6 , and ent_6 is a husband to ent_20 , then ent_3 is the daughter of ent_20']}

In [5]:
split = "bwd_short"
proof_type = "short"
direction = "backward"

In [8]:
list(range(2,7,2))

[2, 4, 6]

In [22]:
"<story> ent_3 is the sister of ent_6 . ent_20 is ent_3 's son . <query> how are ent_6 and ent_20 related to each other ? <proof> since ent_20 is a son to ent_3 , and ent_3 is a sister to ent_6 , then ent_20 is a nephew to ent_6 <answer> ent_20 is the nephew of ent_6 .".replace("<query>", "[query]")

"<story> ent_3 is the sister of ent_6 . ent_20 is ent_3 's son . [query] how are ent_6 and ent_20 related to each other ? <proof> since ent_20 is a son to ent_3 , and ent_3 is a sister to ent_6 , then ent_20 is a nephew to ent_6 <answer> ent_20 is the nephew of ent_6 ."

In [28]:
from pathlib import Path
train = []
for i in range(2,7,2):
    with open(Path(direction) / "train" / f"{proof_type}_proof_1.{i}_train_facts_anon.txt.4000", encoding="utf8") as f:
        for l in f:
            l = l.strip()
            l = l.replace("<query>", "[query]")
            d = process_instance(l)
            l = l.replace("<QUERY>", "[query]").lower()
            assert l.find(d["source"].lower()) != -1, f"line:{l} Parsed:{d['source'].lower()}"
            assert l.find(d["target"].lower()) != -1
            scratchpad = " . ".join(d["scratchpad"])
            assert l.find(scratchpad.lower()) != -1, (scratchpad, l)
            d["category"] = f"{i}"
            train.append(d)

train[0],len(train)

({'source': "ent_3 is the sister of ent_6 . ent_20 is ent_3 's son . [query] How are ent_6 and ent_20 related to each other ?",
  'target': 'ent_20 is the nephew of ent_6',
  'scratchpad': ['since ent_20 is a son to ent_3 , and ent_3 is a sister to ent_6 , then ent_20 is a nephew to ent_6'],
  'category': '2'},
 276904)

In [29]:
from pathlib import Path
valid = []
for i in range(2,7,2):
    with open(Path(direction) / "valid" / f"{proof_type}_proof_1.{i}_valid_facts_anon.txt.4000") as f:
        for l in f:
            l = l.strip()
            d = process_instance(l)
            l = l.replace("<QUERY>", "[query]").lower()
            assert l.find(d["source"].lower()) != -1
            assert l.find(d["target"].lower()) != -1
            scratchpad = " . ".join(d["scratchpad"][1:-1])
            assert l.find(scratchpad.lower()) != -1, (scratchpad, l)
            d["category"] = f"{i}"
            valid.append(d)

valid[0],len(valid)

({'source': 'ent_6 is a uncle of ent_3 . ent_3 is the son of ent_20 . [query] How is ent_20 related to ent_6 ?',
  'target': 'ent_6 is the brother of ent_20',
  'scratchpad': ['since ent_6 is a uncle to ent_3 , and ent_3 is a son of ent_20 , then ent_6 is a brother to ent_20'],
  'category': '2'},
 31346)

In [30]:
from pathlib import Path
test = []
for i in range(2, 11):
    with open(Path(direction) / "test" / str(i) / f"{proof_type}_proof_1.{i}_test_facts_ANON.txt") as f:
        for l in f:
            l = l.strip()
            d = process_instance(l)
            d["category"] = f"{i}"
            l = l.replace("<QUERY>", "[query]").lower()
            assert l.find(d["source"].lower()) != -1
            assert l.find(d["target"].lower()) != -1
            scratchpad = " . ".join(d["scratchpad"][1:-1])
            assert l.find(scratchpad.lower()) != -1, (scratchpad, l)
            test.append(d)

test[0], len(test)

({'source': 'ent_3 is a daughter of ent_6 . ent_6 is a husband to ent_20 . [query] How is ent_20 related to ent_3 ?',
  'target': 'ent_3 is the daughter of ent_20',
  'scratchpad': ['since ent_3 is a daughter of ent_6 , and ent_6 is a husband to ent_20 , then ent_3 is the daughter of ent_20'],
  'category': '2'},
 77288)

In [36]:
from collections import Counter
Counter([d["category"] for d in train])

Counter({'2': 96011, '4': 89971, '6': 90922})

In [37]:
from collections import Counter
Counter([d["category"] for d in valid])

Counter({'2': 10971, '4': 10085, '6': 10290})

In [38]:
from collections import Counter
Counter([d["category"] for d in test])

Counter({'2': 3102,
         '3': 10054,
         '4': 9945,
         '5': 9525,
         '6': 8788,
         '7': 7560,
         '8': 8805,
         '9': 9608,
         '10': 9901})

In [31]:
import jsonlines

def save_jsonl(ds_dir: Path, examples, name):
    with open(ds_dir / f"{name}.jsonl", "w") as f:
        jsonlines.Writer(f).write_all(examples)

ds_dir = Path(f"clutrr-{split}") / "clutrr" / split
ds_dir.mkdir(parents=True, exist_ok=True)

save_jsonl(ds_dir, train, "train")
save_jsonl(ds_dir, valid, "validation")
save_jsonl(ds_dir, test, "test")

In [33]:
for i, d in enumerate(train):
    if len(d["scratchpad"]) <= 1:
        print(d)
        print(i)
        break

{'source': "ent_3 is the sister of ent_6 . ent_20 is ent_3 's son . [query] How are ent_6 and ent_20 related to each other ?", 'target': 'ent_20 is the nephew of ent_6', 'scratchpad': ['since ent_20 is a son to ent_3 , and ent_3 is a sister to ent_6 , then ent_20 is a nephew to ent_6'], 'category': '2'}
0


In [34]:
! tree {ds_dir}

clutrr-bwd_short/clutrr/bwd_short
├── test.jsonl
├── train.jsonl
└── validation.jsonl

0 directories, 3 files


In [35]:
! WANDB_ENTITY=kzmnjd WANDB_PROJECT=len_gen WANDB_RUN_GROUP=ds_upload wandb artifact put -n data-clutrr-{split} clutrr-{split}

wandb: Uploading directory clutrr-bwd_short to: "kzmnjd/len_gen/data-clutrr-bwd_short:latest" (dataset)
wandb: Adding directory to artifact (./clutrr-bwd_short)... Done. 1.2s
wandb: Currently logged in as: kzmnjd. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.13.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.13.7
wandb: Run data is saved locally in /Users/amirhosein/Development/PycharmProjects/len_gen/scripts/dataset_builders/SGinPG/data/wandb/run-20230219_164413-2bmf1fmm
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fiery-cosmos-2503
wandb: ⭐️ View project at https://wandb.ai/kzmnjd/len_gen
wandb: 🚀 View run at https://wandb.ai/kzmnjd/len_gen/runs/2bmf1fmm
Artifact uploaded, use this artifact in a run by adding:

    artifact = run.use_artifact("kzmnjd/len_gen/data-clutrr-bwd_short:latest")

wandb: Waiting for W&B process to finish... (success).
wandb: Synced fiery-cosm